# **Rejected Response Inference**

## **Config**

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"  # sesuaikan dengan GPU yang tersedia

seed = 42
DATASET_PATH    = "../datasets_full_training_and_test/cqa_full_training_prompt_completion.jsonl"
CACHE_FOLDER    = "../model_cache"
SFT_MODEL_NAMES = {
    "Meta-Llama-3.1-8B": "../model_cache/Meta-Llama-3.1-8B",
    "Aya-23-8B":         "../model_cache/Aya-23-8B",
    "SeaLLMs-v3-7B":     "../model_cache/SeaLLMs-v3-7B",
    "SEA-LION-v3-8B":    "../model_cache/SEA-LION-v3-8B",
    "Sahabat-AI-8B":     "../model_cache/Sahabat-AI-8B"
}
BATCH_SIZE      = 32

## **Import Libraries**

In [2]:
import gc
import time
import json
import random
import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    set_seed
)

pd.set_option("display.max_colwidth", None)

/raid/home/llmsosmed/rlaif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## **Utility Functions**

In [3]:
def set_global_seed(s: int = seed):
    random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    set_seed(s)

def load_sft_model_and_tokenizer(model_path: str, tokenizer_path: str, hf_token: str = None):
    set_global_seed()
    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path,
        use_auth_token=hf_token,
        local_files_only=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=quant_config,
        low_cpu_mem_usage=True,
        use_auth_token=hf_token,
        local_files_only=True
    )
    model.config.use_cache = False
    model.eval()
    return tokenizer, model

def generate_sft_batch(tokenizer, model, prompts, max_new_tokens=256):
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1792
    ).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return [
        text[len(prompt):].split("\n", 1)[0]
        for text, prompt in zip(decoded, prompts)
    ]


In [4]:
set_global_seed()

## **Load Dataset**

In [5]:
df = pd.read_json(DATASET_PATH, lines=True)
print(f"Total examples: {len(df)}")
df.sample(3)

Total examples: 3048


,prompt,completion
1582,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 1 Dalam Peraturan Otoritas Jasa Keuangan ini yang dimaksud dengan: 1. Bank adalah bank umum sebagaimana dimaksud dalam Undang-Undang mengenai perbankan, termasuk kantor cabang dari bank yang berkedudukan di luar negeri, serta bank umum syariah dan unit usaha syariah sebagaimana dimaksud dalam Undang-Undang mengenai perbankan syariah.\n\n2. Direksi adalah organ perseroan yang berwenang dan bertanggung jawab penuh atas pengurusan untuk kepentingan perseroan, sesuai dengan maksud dan tujuan perseroan serta mewakili perseroan, baik di dalam maupun di luar pengadilan sesuai dengan ketentuan anggaran dasar bagi Bank yang berbadan hukum perseroan terbatas atau pemimpin kantor cabang dan pejabat satu tingkat di bawah pemimpin kantor cabang bagi kantor cabang dari bank yang berkedudukan di luar negeri. 3. Dewan Komisaris adalah organ perseroan yang bertugas melakukan pengawasan secara umum dan/atau khusus sesuai dengan anggaran dasar serta memberi nasihat kepada Direksi bagi Bank yang berbadan hukum perseroan terbatas atau organ atau pihak yang ditunjuk untuk melaksanakan fungsi pengawasan bagi kantor cabang dari bank yang berkedudukan di luar negeri. 4. Dewan Pengawas Syariah adalah dewan yang bertugas memberikan nasihat dan saran kepada Direksi serta mengawasi kegiatan Bank agar sesuai dengan prinsip syariah. 5. Komite Audit adalah komite yang dibentuk oleh dan bertanggung jawab kepada Dewan Komisaris dalam membantu melaksanakan tugas dan fungsi Dewan Komisaris. 6. Audit Intern adalah kegiatan pemberian keyakinan dan konsultasi yang bersifat independen dan objektif, dengan tujuan untuk meningkatkan nilai dan memperbaiki operasional Bank, melalui pendekatan yang sistematis, dengan cara mengevaluasi dan meningkatkan efektivitas manajemen risiko, pengendalian, dan proses tata kelola Bank. 7. Standar Profesional Audit Intern adalah standar pelaksanaan audit intern secara profesional sebagaimana ditetapkan oleh asosiasi profesi audit intern, termasuk pedoman pelaksanaan standar.\n\nQuestion: Apa pengertian Audit Intern menurut peraturan ini?\nAnswer:","Audit Intern adalah kegiatan pemberian keyakinan dan konsultasi yang bersifat independen dan objektif, dengan tujuan untuk meningkatkan nilai dan memperbaiki operasional Bank, melalui pendekatan yang sistematis, dengan cara mengevaluasi dan meningkatkan efektivitas manajemen risiko, pengendalian, dan proses tata kelola Bank."
2669,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 2 BPRS wajib menyediakan modal minimum yang dihitung dengan menggunakan rasio KPMM paling rendah sebesar 12% (dua belas persen) dari ATMR sejak 1 Januari 2020. Pasal 3 (1) Modal terdiri atas: a. modal inti (tier 1) yang meliputi : 1. modal inti utama; 2. modal inti tambahan; dan b. modal pelengkap (tier 2). (2) Modal pelengkap sebagaimana dimaksud pada ayat (1) huruf b hanya dapat diperhitungkan paling tinggi sebesar 100% (seratus persen) dari modal inti.\n\nQuestion: Apa kategori penilaian untuk modal pelengkap?\nAnswer:",Saya tidak tahu terkait kategori penilaian untuk modal pelengkap.
2091,"Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”\n\nContext:\nPasal 3 Ruang Lingkup Layanan Pengaduan terdiri atas: a. penerimaan Pengaduan; b. penanganan Pengaduan; dan c. penyelesaian Pengaduan. Bagian Ketiga Prinsip Layanan Pengaduan Pasal 4 PUJK dilarang mengenakan biaya Layanan Pengaduan kepada Konsumen. Pasal 5 (1) PUJK wajib memiliki prosedur secara tertulis mengenai Layanan Pengaduan. (2) PUJK wajib mencantumkan prosedur singkat Layanan Pengaduan se

## **Generate Rejected Responses & Build Preference Pairs**

In [ ]:
# kita pakai satu SFT model (atau bisa loop jika mau multi-model)
for model_key, sft_path in SFT_MODEL_NAMES.items():
    print(f"\n▶ Generating rejected responses with SFT {model_key}")
    tokenizer_dir = os.path.join(CACHE_FOLDER, model_key)
    tokenizer, model = load_sft_model_and_tokenizer(
        sft_path, tokenizer_dir, hf_token=os.getenv("HF_TOKEN")
    )

    output_path = f"preference_{model_key}.jsonl"

    # warm-up…
    _ = model.generate(
        **tokenizer("Warm up", return_tensors="pt").to(model.device),
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id
    )

    records = []
    start = time.time()
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc=f"Inferring {model_key}"):
        batch   = df.iloc[i : i + BATCH_SIZE]
        prompts = batch["prompt"].tolist()
        preds   = generate_sft_batch(tokenizer, model, prompts)

        for prompt, chosen, rejected in zip(prompts, batch["completion"], preds):
            records.append({
                "prompt":   prompt,
                "chosen":   chosen,
                "rejected": rejected
            })

    print(f"→ Inference time: {time.time()-start:.1f}s, total pairs: {len(records)}")

    # simpan raw preference file per model
    with open(output_path, "w", encoding="utf-8") as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"✔️ Saved raw preference dataset to `{output_path}`")

    # cleaning duplicates
    df_pref = pd.read_json(output_path, lines=True)
    mask_same = df_pref["chosen"].str.strip() == df_pref["rejected"].str.strip()
    df_clean = df_pref[~mask_same].reset_index(drop=True)
    clean_path = output_path.replace(".jsonl", "_clean.jsonl")
    df_clean.to_json(clean_path, orient="records", lines=True, force_ascii=False)
    print(f"✔️ Cleaned preference dataset saved to `{clean_path}`")

    # tampil sampel
    display(df_clean.sample(5).reset_index(drop=True))

    # selesai untuk model ini
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()


▶ Generating rejected responses with SFT Sahabat-AI-8B
[2025-06-08 07:58:52,071] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Inferring Sahabat-AI-8B:  14%|█▎        | 13/96 [2:08:26<15:02:32, 652.44s/it]

In [ ]:
import signal

os.kill(os.getpid(), signal.SIGTERM)